In [1]:
import pandas as pd
import numpy as np

from autoencoder import ToyDetectors, DEDXEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow.keras.backend as K

Welcome to JupyROOT 6.10/08


In [2]:
TD = ToyDetectors()
DFTD = TD.GenerateToyParticles()

In [3]:
DFTD.head()

,BBA,BBS,ITS,TOF,TPCROC0,TPCROC1,TPCROC2,TRD,beta,pmeas,p,particle
0,1.589041,0.002927,0.003327,1.009881,1.341752,1.551180,1.518810,1.504835,1.0,5.181740,5.156273,0
1,1.588907,0.002855,0.002994,0.992257,1.420755,1.675755,1.528528,1.532489,1.0,3.277865,3.234610,0
2,1.589086,0.002979,0.003189,0.992817,1.846851,1.319379,1.343102,1.448156,1.0,7.168424,7.266984,0
3,1.589105,0.003022,0.003048,1.002209,1.691059,1.355429,1.559598,1.658196,1.0,9.369128,9.583644,0
4,1.588854,0.002839,0.002559,1.010613,1.529031,1.644516,1.520881,1.324857,1.0,2.884021,2.901373,0


In [4]:
DEDXEnc = DEDXEncoder()

In [5]:
DEDXEnc.CreateModel()

In [6]:
DEDXEnc.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           512         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           4160        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           4160        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (D

In [7]:
train, test =train_test_split(DFTD, test_size=0.5)
train_data = train[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]
test_data  = test[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]

train_fdata= train[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD"]]
test_fdata = test[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD"]]

In [8]:
scaler = StandardScaler()
scaler.fit(train[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]])


StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
DEDXEnc.model.fit(scaler.transform(train_data),train_fdata, epochs=12, batch_size=128, 
          validation_data=[scaler.transform(test_data),test_fdata])

Train on 40000 samples, validate on 40000 samples
Epoch 1/12
40000/40000 [==============================] - 4s 106us/step - loss: 0.1405 - mean_squared_error: 0.2848 - val_loss: 0.0389 - val_mean_squared_error: 0.0718
Epoch 2/12
40000/40000 [==============================] - 3s 74us/step - loss: 0.0344 - mean_squared_error: 0.0610 - val_loss: 0.0307 - val_mean_squared_error: 0.0525
Epoch 3/12
40000/40000 [==============================] - 3s 73us/step - loss: 0.0291 - mean_squared_error: 0.0486 - val_loss: 0.0253 - val_mean_squared_error: 0.0418
Epoch 4/12
40000/40000 [==============================] - 3s 74us/step - loss: 0.0248 - mean_squared_error: 0.0405 - val_loss: 0.0217 - val_mean_squared_error: 0.0356
Epoch 5/12
40000/40000 [==============================] - 3s 74us/step - loss: 0.0206 - mean_squared_error: 0.0327 - val_loss: 0.0329 - val_mean_squared_error: 0.0592
Epoch 6/12
40000/40000 [==============================] - 3s 74us/step - loss: 0.0165 - mean_squared_error: 0.0266

2019-02-08 13:02:36.256888: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: FMA


In [13]:
DEDXEnc.BGfromModel([train[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]])

[array([[8.072622],
        [8.430906],
        [8.537409],
        ...,
        [8.547984],
        [8.530241],
        [8.20077 ]], dtype=float32)]

In [14]:
out=DEDXEnc.model.predict(scaler.transform(test_data))
AE_predict = pd.DataFrame(out)
AE_predict.columns = ["ITS_ae", "TOF_ae", "TPCROC0_ae", "TPCROC1_ae", "TPCROC2_ae", "TRD_ae"]
ogb=DEDXEnc.BGfromModel([(scaler.transform(test_data))])
#print(ogb[-1])
GB_predict = pd.DataFrame(ogb[-1])
GB_predict.columns = ["gb_ae"]
test = test.reset_index()
df_test = pd.concat([test,AE_predict,GB_predict], axis = 1)
df_test.head()

,index,BBA,BBS,ITS,TOF,TPCROC0,TPCROC1,TPCROC2,TRD,beta,pmeas,p,particle,ITS_ae,TOF_ae,TPCROC0_ae,TPCROC1_ae,TPCROC2_ae,TRD_ae,gb_ae
0,78992,3.113516,0.005475,0.006046,0.448106,3.149678,2.648403,2.973634,3.228630,0.445178,0.476006,0.466336,4,0.005392,0.449152,3.068460,3.068460,3.068460,3.068460,-0.687736
1,70851,3.892440,0.006902,0.005326,0.387304,3.670860,4.402737,3.556867,4.557834,0.388496,0.392972,0.395473,4,0.006915,0.388074,3.899369,3.899369,3.899369,3.899369,-0.864945
2,9746,1.589074,0.002962,0.002236,0.985866,1.566059,1.367802,1.659534,1.776052,1.000000,6.470735,6.470266,0,0.002328,0.999988,1.497916,1.497916,1.497916,1.497916,5.334323
3,76343,2.351893,0.004089,0.004371,0.526751,2.186766,2.440902,2.099237,2.164081,0.528284,0.580809,0.583616,4,0.004100,0.527508,2.357556,2.357556,2.357556,2.357556,-0.476544
4,11050,1.589094,0.002995,0.003014,1.007747,1.603374,1.651585,1.622795,1.511636,1.000000,8.141613,8.053390,0,0.002521,0.999999,1.574637,1.574637,1.574637,1.574637,6.566881
